<a href="https://colab.research.google.com/github/ucfilho/marquesgabi/blob/master/Mahotas_Fracionado_teste_003_nov_11_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mahotas

     |████████████████████████████████| 1.5MB 2.6MB/s 
  Created wheel for mahotas: filename=mahotas-1.4.8-cp36-cp36m-linux_x86_64.whl size=4263966 sha256=3724e4c6d7cd22abc782187f69be7e6b5129cce647fb3d6a52cd63661946df3d
  Stored in directory: /root/.cache/pip/wheels/3e/8b/93/690deca49a46242df274db70dbef26c0a6fa86e378b677e1ab
Successfully built mahotas


In [0]:
# proximas etapas que falta : dividir a foto em 10x10 e aplicar o mahotas em cada foto
# encontrar 
# GLCM?
# COLOCAR MAIS VARIAVEIS NAS FOTOS
# VER QUAIS PARAMETROS USAR PARA CALCULAR O PSD (particle size distribution)
# GERAR AS IMAGENS ARTIFICIAIS

In [0]:
import numpy as np
import cv2
from skimage import data
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from skimage import io, util
import mahotas.features.texture as mht
import mahotas.features
import os 
import zipfile
from skimage import filters
from skimage import exposure
import skimage.feature as sk
from google.colab import files
from numpy import linalg as LA
%matplotlib inline

In [0]:
!wget http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
#!ls -l trash_nov_22_2018.zip

--2019-11-11 16:08:05--  http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19584173 (19M) [application/zip]
Saving to: ‘Sugar_set_30_2019.zip’

Sugar_set_30_2019.z 100%[===================>]  18.68M  4.14MB/s    in 4.8s    

2019-11-11 16:08:13 (3.88 MB/s) - ‘Sugar_set_30_2019.zip’ saved [19584173/19584173]



In [0]:
# https://buildmedia.readthedocs.org/media/pdf/mahotas/latest/mahotas.pdf

# Why there are only 13 features implemented? (pag 42)
# Only the first 13 features are implemented. The last (14th) feature is 
# normally considered to be unstable, although it
# is not clear to me why this is. (See this unanswered question on Cross-validated).

# What mahotas.features.haralick return as answer?  (pag 58)
# mahotas.features.haralick returns features in 4 directions. 
# We just take the mean (sometimes you use the spread ptp() too).

# Who uses mahotas? (pag 62)
# In June 2016, there were 34 papers citing the mahotas paper




In [0]:
file_name = zipfile.ZipFile('Sugar_set_30_2019.zip', 'r')
file_name.extractall()

In [0]:
# all images are resized in variable x, are in gray in variable p and in variable w are dataframe
FILE='Sugar_set_30_2019.zip'
img_name=[]
x=[]
p=[]
w=[]
Size=64 # tamanho da foto
Sub_Size=4 # tamanho do fracionamento
Row_Crop=1/2 # posicao do corte
Crop=int(Size*Row_Crop)

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)
        #x.append(cv2.imread(name))
        x.append(cv2.resize(cv2.imread(name),(Size,Size)))

nrow=len(img_name)
ncol=Size*Size
pp=np.zeros((nrow,ncol))
for i in range(nrow):
    w.append(cv2.cvtColor(np.array(x[i]), cv2.COLOR_BGR2GRAY))
    pp[i,:]=w[i].ravel()

In [0]:
Nomes=['ASM','constrast','correl','variance','inv diff mom','sum aveg',
       'sum var','sum entropy','entropy','dif var','dif entropy','IMC1',
       'IMC2']


In [0]:
Nfotos=len(img_name)
GLCM=[]
glcm_haralick=[]
for k in range(Nfotos):
  p=pp[k,:].reshape(Size,Size)
  Mahotas =pd.DataFrame(mahotas.features.haralick(p.astype(int)), columns =Nomes)
  GLCM=[]
  for i in Nomes:
    GLCM.append(Mahotas[i].mean())
  glcm_haralick.append(GLCM)

print(pd.DataFrame(glcm_haralick,columns=Nomes))


         ASM    constrast    correl    variance  inv diff mom    sum aveg  \
0   0.028926    95.465625  0.007925   47.260693      0.100675  224.453125   
1   0.032441  1019.881250 -0.090186  474.659844      0.089655  233.937500   
2   0.029375   176.337500  0.146018  103.655449      0.090540  226.468750   
3   0.030664    95.121875 -0.002434   47.590459      0.101340  228.946875   
4   0.029590   159.746875  0.044044   83.721436      0.087985  222.703125   
5   0.029727   104.781250  0.201476   65.540625      0.097444  229.425000   
6   0.030840   102.478125  0.129225   58.843154      0.128326  231.996875   
7   0.032656    97.018750  0.329610   73.599180      0.130997  235.306250   
8   0.030488   128.940625  0.274450   88.978857      0.097787  251.615625   
9   0.029551   174.531250  0.074583   95.730527      0.125745  248.462500   
10  0.030078    71.443750  0.278276   49.246641      0.123552  254.900000   
11  0.028438  1896.446875 -0.182054  800.328467      0.062714  259.003125   